In [ ]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
from estival.sampling import tools as esamp
from tbdynamics.calibration.utils import plot_covid_scenarios_comparison
from tbdynamics.inputs import load_targets
from tbdynamics.constants import params_name, quantiles
import arviz as az
from tbdynamics.calibration.utils import get_bcm
import numpy as np

In [ ]:
OUT_PATH = Path.cwd() / 'runs/r2508'

In [ ]:

spaghetti = pd.read_hdf(OUT_PATH / 'results.hdf', 'spaghetti')
quantile_outputs = esamp.quantiles_for_results(spaghetti, quantiles)
targets = load_targets()

In [ ]:
# plot_spaghetti(spaghetti, ['total_population','notification'], 2, targets)

In [ ]:
# plot_spaghetti(spaghetti, ['prevalence_pulmonary','incidence'], 2, targets)

In [ ]:
# plot_output_ranges(quantile_outputs,targets, ['total_population','notification','adults_prevalence_pulmonary'], 1, 2010, 2025)

In [ ]:
# plot_output_ranges(quantile_outputs,targets, ['incidence'], quantiles, 1, 2010, 2025)

In [ ]:
# plot_output_ranges(quantile_outputs,targets, ['mortality_raw'], quantiles, 1, 2010, 2025)

In [ ]:
idata = az.from_netcdf(OUT_PATH / 'calib_full_out.nc')

In [ ]:
# idata = idata.sel(chain = [0,1,2,3], draw=np.s_[50000:])
# idata = idata.sel(chain = [0,1,2,3], draw=np.s_[50000:])

In [ ]:
# az.plot_trace(idata, figsize=(16,3.1*len(idata.posterior)))

In [ ]:
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':True
}

In [ ]:
# priors = get_bcm(params, covid_effects).priors
# req_vars = list(priors.keys())

In [ ]:
# tabulate_calib_results(idata, params_name)

In [ ]:
plot_covid_scenarios_comparison(params, idata, ['incidence', 'mortality_raw'],[2021,2022],quantiles,1)